## IMPORT

In [ ]:
!pip install --upgrade pip
!pip install gradio
!pip install --upgrade transformers sentencepiece datasets[audio]
!pip install lmqg
!pip install --upgrade evaluate

  Using cached typer-0.12.3-py3-none-any.whl.metadata (15 kB)
Using cached typer-0.12.3-py3-none-any.whl (47 kB)
  Attempting uninstall: typer
    Found existing installation: typer 0.9.4
    Uninstalling typer-0.9.4:
      Successfully uninstalled typer-0.9.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.7.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.
weasel 0.3.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.
  Using cached typer-0.9.4-py3-none-any.whl.metadata (14 kB)
Using cached typer-0.9.4-py3-none-any.whl (45 kB)
  Attempting uninstall: typer
    Found existing installation: typer 0.12.3
    Uninstalling typer-0.12.3:
      Successfully uninstalled typer-0.12.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behav

In [ ]:
!pip install rouge_score

In [ ]:
!pip install --upgrade transformers sentencepiece datasets[audio]

In [ ]:
import gradio as gr
import librosa
import numpy as np
import lmqg
import evaluate
import rouge_score
from pprint import pprint
from lmqg import TransformersQG
from transformers import pipeline

In [ ]:
model = TransformersQG(language='en', model='lmqg/t5-base-squad-qg-ae')

In [ ]:
MODEL_NAME = "openai/whisper-small"
BATCH_SIZE = 8

pipe = pipeline(
    task="automatic-speech-recognition",
    model=MODEL_NAME,
    chunk_length_s=30,
)


# Copied from https://github.com/openai/whisper/blob/c09a7ae299c4c34c5839a76380ae407e7d785914/whisper/utils.py#L50
def format_timestamp(seconds: float, always_include_hours: bool = False, decimal_marker: str = "."):
    if seconds is not None:
        milliseconds = round(seconds * 1000.0)

        hours = milliseconds // 3_600_000
        milliseconds -= hours * 3_600_000

        minutes = milliseconds // 60_000
        milliseconds -= minutes * 60_000

        seconds = milliseconds // 1_000
        milliseconds -= seconds * 1_000

        hours_marker = f"{hours:02d}:" if always_include_hours or hours > 0 else ""
        return f"{hours_marker}{minutes:02d}:{seconds:02d}{decimal_marker}{milliseconds:03d}"
    else:
        # we have a malformed timestamp so just return it as is
        return seconds


def transcribe(file):
    outputs = pipe(file, batch_size=BATCH_SIZE)
    text = outputs["text"]
    return text

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import torch
import soundfile as sf
from datasets import load_dataset
import IPython.display as ipd

# Initialize processor, model, and vocoder
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model_tts = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

# Load speaker embeddings from a dataset
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)


In [ ]:
import gradio as gr
import logging

questions = []
current_question = 0
answers = []
score = 0
sentiment_label = None

logging.basicConfig(level=logging.DEBUG)

def from_text_to_speech(text):
    # Input text for speech synthesis
    inputs = processor(text=text, return_tensors="pt")

    # Generate speech
    speech = model_tts.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

    # Save the audio to a temporary file
    audio_path = "/tmp/question_audio.wav"
    sf.write(audio_path, speech.numpy(), samplerate=16000)

    # Return the audio file path for Gradio
    return audio_path

def load_data(input_data):
    global questions, answers, score, current_question  # Use the global variables
    try:
        context = input_data
        # model prediction
        question_answer = model.generate_qa(context)  # Assuming model has a generate_qa method
        #question_answer = [(i, i) for i in range(1,5)]
        # Separate questions and answers
        questions = []
        answers = []
        score = 0
        current_question = 0
        for question, answer in question_answer:
            questions.append(question)
            answers.append(answer)
        return "Ready to Play!"
    except Exception as e:
        return f"Training failed: {str(e)}"

def turn(audio_data, text_data):
    logging.debug(f'audio_data: {audio_data}, text_data: {text_data}')
    global questions, answers, score, current_question


    def check_answer(user_answer, correct_answer):
        rouge = evaluate.load('rouge')  # Carica ROUGE
        references = [correct_answer]  # Risposta corretta
        predictions = [user_answer]  # Risposta dell'utente

        results = rouge.compute(predictions=predictions, references=references)
        rouge_l_f1 = results['rougeL']  # F1 score per ROUGE-L

        # Restituisci il valore di somiglianza calcolato
        return rouge_l_f1

    if(audio_data is not None):
      user_answer = transcribe(audio_data)
    elif (text_data is not None):
      user_answer = str(text_data)
    else:
      user_answer = ""
    correct_answer_to_show = answers[current_question]

    if current_question <= len(questions):
        score = check_answer(user_answer, answers[current_question])
        current_question += 1
        return score, correct_answer_to_show
    else:
        return "","Game finished!"




def show_new_question():
    global current_question, questions
    if current_question >= len(questions):
        return "Game Finished!"
    return questions[current_question]

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(
    """
    # AI Study
    AI Study is a web application which goal is to combine AI tools and proven cognitive tactics to guaranteeing durable and usable knowledge which won’t fade away as soon as the exam is over.
    <br>The program is designed based on the main learning techniques:
      - Feynman Technique
      - Active Recall
      - Space Repetition
    These developed with a Gamification perspective to achieved your progress.

    How to Use:
      1. Upload the materials [pdf,text,audio]
      2. Select your interlocutor, it will define the difficulty of the study session
      3. Click on the 'Submit' button. <strong>Voila!</strong>
      4. The platform will generate a question for you...
      5. Answer your question [text,audio] and wait for a feedback

      Created by PizzAIoli :)

    """)

    with gr.Tab("Data Loading"):
        input_text = gr.Textbox(label = "input")
        btn = gr.Button("Upload text")
        show_status = gr.Textbox(label = "Show status")
        btn.click(fn=load_data, show_progress=True, inputs=input_text, outputs=[show_status])


    with gr.Tab("Game"):
        new_question_text = gr.Textbox(label=" ")
        speak_btn = gr.Button("Speak Question")
        speak_btn.click(from_text_to_speech, inputs=[new_question_text], outputs=gr.Audio(type="filepath"))
        audio = gr.Audio(type="filepath")
        text = gr.Textbox()
        submit_btn = gr.Button("Submit Answer")


        with gr.Row():
          score_text = gr.Label(label="Score")
          answer_correct_text = gr.Label(label="Correct Answer")
        submit_btn.click(turn, inputs=[audio, text], outputs=[score_text, answer_correct_text])

        new_question_btn = gr.Button("New Question")
        new_question_btn.click(show_new_question, outputs=new_question_text)


demo.launch(debug = True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://8943283a150cd076d4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


100%|██████████| 5/5 [00:00<00:00, 457.70it/s]
